### Import Requirements

In [ ]:
import pandas as pd
import json
import os

### From local json's to Pandas DataFrame

In [ ]:
path = 'storage/current/20231223/'
json_names = os.listdir(path)
jsons = []
for name in json_names:
    json_path = path+name
    with open(json_path,'r') as f:
        jsons.append(json.load(f))
# Create dataframe from jsons
df = pd.json_normalize(jsons)
# Rename columns to be identifiable
df.columns = ['City','Region','Country','Lat','Long','TZ_ID','Localtime_Epoch', \
             'Localtime','Last_updated_epoch','Last_Updated','Temp_C','Temp_F', \
             'Is_day','Condition.Text','Condition.Icon','Condition.Code', \
             'Wind_mph','Wind_kph','Wind_degree','Wind_dir','Pressure_mb', \
             'Pressure_in','Precip_mm','Precip_in','Humidity','Cloud','Feelslike_c', \
             'Feeslike_f','Vis_km','Vis_miles','UV','Gust_mph','Gust_kph']


### Analyse DataFrame structure to identify required transformation

In [ ]:
# Look for nan values
print(df.isna().any())
#Verify current dtypes and determine any transformation required
print(df.dtypes)

### Removing redundant data

In [ ]:
df.dtypes
location = ['City','Region','Country','Lat','Long','TZ_ID','Localtime','Last_Updated']
weather = ['Condition.Text','Condition.Code','Temp_C','Wind_kph','Wind_degree','Wind_dir','Pressure_in','Precip_mm','Humidity','Cloud','Feelslike_c','Vis_km','UV','Gust_kph']

df[location+weather].head()

### Transformations required:
- Datetime from strings (Localtime and Last_Updated)
- If data is not identifiable (null City or null Lat or Long), drop full row
- If any of those is null but is still identifiable, complete it with info in the DB
- Accept Null values for any other field since the data can still be used

### Astronomy data 

In [ ]:
path = 'storage/astronomy/20231223/'
json_names = os.listdir(path)
jsons = []
for name in json_names:
    json_path = path+name
    with open(json_path,'r') as f:
        jsons.append(json.load(f))
# Create dataframe from jsons
df = pd.json_normalize(jsons)
df.columns = ['City','Region','Country','Lat','Long','TZ_ID','Localtime_epoch','Localtime',\
              'Sunrise','Sunset','Moonrise','Moonset','Moon_Phase','Moon_Ilumination','Moon_Up','Moon_Down']
df.dtypes

# If not identifiable, drop row
# Transform in datetime Localtime, Sunrise, Sunset, Moonrise, Moonset

### Removing redundant data

In [ ]:
location = ['City','Region','Country','Lat','Long','TZ_ID','Localtime'] 
astro = ['Sunrise','Sunset','Moonrise','Moonset','Moon_Phase','Moon_Ilumination','Moon_Up','Moon_Down']
df[location+astro].head()

### Transformations required:
- If not identifiable, drop row
- Datetime from strings for : Localtime, Sunrise, Sunset, Moonrise and Moonset
- ...

### Marine Data

In [ ]:
path = 'storage/marine/20231223/'
json_names = os.listdir(path)
jsons = []
for name in json_names:
    json_path = path+name
    with open(json_path,'r') as f:
        jsons.append(json.load(f))
# Create dataframe from jsons
locations = [sample['location'] for sample in jsons]
forecasts = [sample['forecast']['forecastday'] for sample in jsons]

forecast_df = pd.DataFrame()
for sample in forecasts:
    current_df = pd.json_normalize(sample)
    forecast_df = pd.concat([forecast_df,current_df],axis=0)
forecast_df = forecast_df.reset_index()
loc_df = pd.json_normalize(locations)

flat_df = pd.concat([loc_df,forecast_df],axis=1)
flat_df.columns = ['City','Region','Country','Lat','Long','TZ_ID','Localtime_epoch','Localtime','Index','Date','Date_Epoch',\
                   'Hourly_data','Maxtemp_C','Maxtemp_F','Mintemp_C','Mintemp_F','Avgtemp_C','Avgtemp_F','Maxwind_mph','Maxwind_kph',\
                    'Totalprecip_mm','Totalprecip_in','Totalsnow_cm','Avgvis_km','Avgvis_miles','Avghumidity','Tides',\
                    'Condition.text','Condition.icon','Condition.code','UV','Sunrise','Sunset','Moonrise','Moonset']

### Removing redundant data

In [ ]:
location = ['City','Region','Country','Lat','Long','TZ_ID','Localtime'] 
marine = ['Date','Hour','Maxtemp_C','Mintemp_C','Avgtemp_C','Maxwind_kph',\
        'Totalprecip_mm','Totalsnow_cm','Avgvis_km','Avghumidity','Tides','Condition.text',\
        'Condition.code','UV','Sunrise','Sunset','Moonrise','Moonset']
flat_df[location+marine].iloc[0].Hour

### Transformations required:
- If not identifiable, drop row
- Datetime from strings for : Date,Localtime, Sunrise, Sunset, Moonrise and Moonset
- ...

### Sports Data

In [64]:
path = 'storage/sports/20231223/'
json_names = os.listdir(path)
jsons = []
for name in json_names:
    json_path = path+name
    with open(json_path,'r') as f:
        jsons.append(json.load(f))
# Create dataframe from jsons
df = pd.json_normalize(jsons)


In [70]:
# football_df = df[(df["football"].str.len() != 0)|(df["cricket"].str.len() != 0)|(df["golf"].str.len() != 0)]
football_df = pd.json_normalize(jsons,record_path='football')
golf_df = pd.json_normalize(jsons,record_path='golf')
cricket_df = pd.json_normalize(jsons,record_path='cricket')
flat_df = pd.concat([football_df,golf_df,cricket_df],axis=0)
flat_df

,stadium,country,region,tournament,start,match
0,Kingstonian Fc,United Kingdom,,Isthmian League,2023-12-26 15:00,Kingstonian vs Chatham Town
1,Slough Town F.C,United Kingdom,,National League South,2023-12-26 13:00,Slough vs Hampton & Richmond
2,Hayes Fc,United Kingdom,,Southern Premier League South,2023-12-26 15:00,Hayes & Yeading vs Hanwell Town
3,Brentford Fc,United Kingdom,,Premier League,2023-12-27 19:30,Brentford vs Wolverhampton Wanderers
4,Aldershot Town,United Kingdom,,National League,2023-12-26 15:00,Aldershot Town vs Woking
5,Chelsea,United Kingdom,,Premier League,2023-12-27 19:30,Chelsea vs Crystal Palace
6,Harrow Borough Fc,United Kingdom,,Southern Premier League South,2023-12-26 15:00,Harrow Borough vs Hendon
7,Maidenhead United Fc,United Kingdom,,National League,2023-12-26 15:00,Maidenhead United vs Wealdstone
8,Crystal Palace,United Kingdom,,Premier League,2023-12-30 15:00,Crystal Palace vs Brentford
9,Dulwich Hamlet,United Kingdom,,Isthmian League,2023-12-26 15:00,Dulwich Hamlet vs Carshalton Athletic


### Transformations required:
- Drop all rows containing only empty lists
- Datetime from strings for : start
- Replace empty string with null (region)